In [1]:
class bcolors: # 4-bit Colours ANSI
    OKMAGENTA = '\033[95m'
    OKBLUE = '\033[94m[SCRAPING]'
    OKGREEN = '\033[92m'
    OKYELLOW = '\033[93m'
    OKCYAN = '\033[96m'
    OKRED = '\033[91m'
    ENDC = '\033[0m'

bc = bcolors

In [2]:
import time
import pandas as pd
from selenium import webdriver

from bs4 import BeautifulSoup
from time import sleep

import warnings 
warnings.filterwarnings('ignore')

In [3]:
df = pd.DataFrame(columns=('brand_name', 'product_name','trans_name','price'))
df

,brand_name,product_name,trans_name,price


In [4]:
import os
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options=options)
cream_url = "https://kream.co.kr/search?category_id=34&per_page=40"
driver.get(cream_url)

In [5]:
from selenium.webdriver.common.by import By
url_source = driver.page_source
soup = BeautifulSoup(markup=url_source, features='html.parser')
brand_name_elements = soup.find_all('p', {'class':'product_info_brand brand'})
product_name_elements = soup.find_all('p', {'class':'name'})
trans_name_elements = soup.find_all('p', {'class':'translated_name'})
price_elements = soup.find_all('p', {'class':'amount'})

In [6]:
for i in range(len(brand_name_elements)):
	brand = str(brand_name_elements[i].text).strip()
	name = str(product_name_elements[i].text).strip()
	trans_name = str(trans_name_elements[i].text).strip()
	price = str(price_elements[i].text).strip()[:-1]
	# img_elements = soup.find_all('img', {'alt':trans_name})
	# img = img_elements[0]['src']
	# print(img_elements)

	df = df.append({'brand_name':brand, 'product_name': name, 'trans_name': trans_name,'price': price}, ignore_index=True)

In [7]:
img_df = pd.DataFrame(columns=('idx', 'product_img'))
end_point = len(brand_name_elements)
for idx in range(end_point):
    shoesPath = '//*[@id="__layout"]/div/div[2]/div[3]/div[2]/div[1]/div[{}]/a'.format(idx+1)
    shoesBtn = driver.find_element(By.XPATH, shoesPath).click()
    sleep(3)

    url_source = driver.page_source
    soup = BeautifulSoup(markup=url_source, features='html.parser')

    img_list = []
    img_elements = list(soup.find_all('img', {'class':'image', 'alt': '상품 이미지', 'fetchpriority':'high'}))
    for img in img_elements:
        img_list.append(img['src'])
    message = ' {}% | Scraping the Channel...'.format(int(idx/end_point*100))
    print('\r' + bc.OKBLUE + message + bc.ENDC, end="")
    
    img_df = img_df.append({'idx':idx, 'product_img':img_list}, ignore_index=True)
    driver.back()
    sleep(3)
print("\n" + bc.OKGREEN + '----------Done!----------'+ bc.ENDC)

[SCRAPING] 97% | Scraping the Channel...
----------Done!----------


In [24]:
df = pd.concat([df, img_df], axis=1)

In [25]:
df = df[['idx', 'brand_name', 'product_name', 'trans_name', 'price', 'product_img']]
df.rename(columns={'idx':'id'}, inplace=True)
df

,id,brand_name,product_name,trans_name,price,product_img
0,0,New Balance,New Balance x Stone Island 574 Legacy Green,뉴발란스 x 스톤 아일랜드 574 레거시 그린,"428,000",[https://kream-phinf.pstatic.net/MjAyMjEyMjhfM...
1,1,Jordan,Jordan 1 High 85 Black White,조던 1 하이 85 블랙 화이트,"369,000",[https://kream-phinf.pstatic.net/MjAyMzAyMDFfM...
2,2,Nike,Nike Dunk Low Retro Grey Fog,나이키 덩크 로우 레트로 그레이 포그,"130,000",[https://kream-phinf.pstatic.net/MjAyMTA5MDlfM...
3,3,Nike,Nike x Ambush Air Force 1 Low SP Phantom,나이키 x 앰부쉬 에어포스 1 로우 SP 팬텀,"257,000",[https://kream-phinf.pstatic.net/MjAyMzAyMTBfM...
4,4,Nike,Nike Air Force 1 '07 Low White,나이키 에어포스 1 '07 로우 화이트,"119,000",[https://kream-phinf.pstatic.net/MjAyMjA2MTVfM...
5,5,Nike,Nike Air Force 1 '07 WB Flax,나이키 에어포스 1 '07 WB 플랙스,"155,000",[https://kream-phinf.pstatic.net/MjAyMjA4MDFfM...
6,6,Nike,Nike x Ambush Air Force 1 Low SP Black,나이키 x 앰부쉬 에어포스 1 로우 SP 블랙,"240,000",[https://kream-phinf.pstatic.net/MjAyMzAyMTBfM...
7,7,Nike,(W) Nike Zoom Vomero 5 Photon Dust and Metalli...,(W) 나이키 줌 보메로 5 포톤 더스트 앤 메탈릭 실버,"202,000",[https://kream-phinf.pstatic.net/MjAyMzAxMzBfM...
8,8,Asics,(W) Asics Gel-Kayano 14 White Midnight,(W) 아식스 젤 카야노 14 화이트 미드나잇,"240,000",[https://kream-phinf.pstatic.net/MjAyMzAxMzFfM...
9,9,New Balance,New Balance 2002R Grey,뉴발란스 2002R 그레이,"145,000",[https://kream-phinf.pstatic.net/MjAyMjAzMzBfM...


In [26]:
df.to_csv('./cream.csv',index = False, encoding='utf-8-sig')